<a href="https://colab.research.google.com/github/NikolaZubic/AppliedGameTheoryHomeworkSolutions/blob/main/domaci6_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ŠESTI DOMAĆI ZADATAK iz predmeta "Primenjena teorija igara" (Applied Game Theory)

a) Simulacija evolutivne igre koja u osnovi ima stratešku igru "Sokola i Goluba". Na kraju se konvergira do evolutivno stabilne strategije (ESS).

# Potrebni import-i

In [ ]:
from random import random
from math import floor
import numpy as np

# Definisanje jedinke (ptice)

In [ ]:
class Bird(object):
    def __init__(self, name, fitness_function):
        """
        Every bird has a name and associated fitness function.

        :param name: {"H", "D"}, Hawk or Dove
        :param fitness_function: fitness function for one individual (particular bird) that belongs to a population
        as a whole
        """
        self.name = name
        self.fitness_function = fitness_function

# Definisanje klase zadužene za simulaciju

In [ ]:
class Simulation(object):
    def __init__(self, population_length, fitness_0, mutation_rate, hawk_probability,
                 number_of_two_bird_encounters_per_generation, fraction_of_the_most_fit_birds, V, C):
        """
        :param population_length: length of the population
        :param fitness_0: initial fitness function for every bird in the population
        :param mutation_rate: probability that a child of HH(DD) mutated to D(H)
        :param hawk_probability: initial ratio of hawks
        :param number_of_two_bird_encounters_per_generation: number of 2-bird contest encounters per generation
        :param fraction_of_the_most_fit_birds: fraction of the most fit birds that we will select for breeding
        :param V: value of a prize that someone gets during an encounter
        :param C: cost for hawk losing encounter to another hawk
        """
        self.population = []
        self.cumulative_fitness_function_for_parents = []
        self.parents_list = []
        self.hawk_ratio_list = []

        self.population_length = population_length
        self.fitness_0 = fitness_0
        self.mutation_rate = mutation_rate
        self.hawk_probability = hawk_probability
        self.number_of_two_bird_encounters_per_generation = number_of_two_bird_encounters_per_generation
        self.fraction_of_the_most_fit_birds = fraction_of_the_most_fit_birds
        self.V = V
        self.C = C

    def initialize_population(self):
        """
        Initializes a fresh population of birds that we use at the very beginning of the simulation. Each bird is
        created as a hawk with "hawk_probability". Every bird has the same initial fitness (self.fitness_0) at birth.

        :return: None, fills self.population collection with birds
        """
        for i in range(0, self.population_length):
            bird_name = "D"
            r = random()
            if r < self.hawk_probability:
                bird_name = "H"

            new_bird = Bird(name=bird_name, fitness_function=self.fitness_0)

            self.population.append(new_bird)

    @property
    def hawk_current_ratio(self):
        """
        Property in Python allows us to interpret this function as an attribute. So, we won't be calling it with
        Simulation.hawk_current_ratio(), but with Simulation.hawk_current_ratio

        :return: ratio of hawks in the whole population
        """
        number_of_hawks = 0

        for i in range(0, self.population_length):
            if self.population[i].name == "H":
                number_of_hawks += 1

        return number_of_hawks / self.population_length

    def encounter(self, bird_1_idx, bird_2_idx):
        """
        Effect of one encounter between bird 1 and bird 2 (represented with their indexes) on their fitness functions.

        :param bird_1_idx: index of first bird
        :param bird_2_idx: index of second bird
        :return: None, fitness function of these birds (one or both) gets updated.
        """
        first_bird = self.population[bird_1_idx].name
        second_bird = self.population[bird_2_idx].name

        if first_bird is "H" and second_bird is "H":
            self.population[bird_1_idx].fitness_function += (self.V - self.C) / 2
            self.population[bird_2_idx].fitness_function += (self.V - self.C) / 2
        elif first_bird is "H" and second_bird is "D":
            self.population[bird_1_idx].fitness_function += self.V
        elif first_bird is "D" and second_bird is "H":
            self.population[bird_2_idx].fitness_function += self.V
        elif first_bird is "D" and second_bird is "D":
            self.population[bird_1_idx].fitness_function += self.V / 2
            self.population[bird_2_idx].fitness_function += self.V / 2

    def search(self, A, i, j, x):
        """
        Searches for index of element of list A in the range [i, j) for which A[index] <= x, but with additional
        condition that A[index + 1] > x (or such that k = j - 1). Basically, a binary search algorithm.

        :param A: list on interval (a, b) where a <= i <= j < b
        :param i: index for the left boundary
        :param j: index for the right boundary
        :param x: value
        :return: index that fulfills these conditions
        """
        if j == i + 1 or j == i:
            return i

        if x < A[floor((i + j) / 2)]:
            return self.search(A, i, floor((i + j) / 2), x)
        else:
            return self.search(A, floor((i + j) / 2), j, x)

    def select_two_distinct_parent_indexes(self):
        """
        Selects two distinct parent indexes. Probability with which we select a parent is proportional to its fitness
        function. This enables us to produce successful species (H or D) with probability that is proportionate to
        their fitness value, not only their current quantity in the population.

        :return: indexes of parents
        """
        r1, r2 = random(), random()
        first_index = self.search(self.cumulative_fitness_function_for_parents, 0,
                                  len(self.cumulative_fitness_function_for_parents), r1)
        second_index = self.search(self.cumulative_fitness_function_for_parents, 0,
                                   len(self.cumulative_fitness_function_for_parents), r2)

        while second_index == first_index:
            r2 = random()
            second_index = self.search(self.cumulative_fitness_function_for_parents, 0,
                                       len(self.cumulative_fitness_function_for_parents), r2)

        return first_index, second_index

    def random_sample_without_replacement(self, N):
        """
        Get a random sample without replacement from interval [0, N)

        :param N: integer number
        :return: sample list
        """
        sample_list = []

        for i in range(0, N):
            sample_list.append(i)

        for i in range(0, N):
            r = random()
            j = floor(r * (N-i))
            x = sample_list[N - 1 - i]
            sample_list[N - 1 - i] = sample_list[j]
            sample_list[j] = x

        return sample_list

    def game(self):
        """
        We define game in such a manner that we get some random sample of indexes and encounter some of the population
        individuals.

        :return: updates fitness functions based on these encounters
        """
        for i in range(0, self.number_of_two_bird_encounters_per_generation):
            population_mix = self.random_sample_without_replacement(self.population_length)

            for j in range(0, floor(self.population_length / 2)):
                self.encounter(population_mix[j], population_mix[j + floor(self.population_length / 2)])

    def produce_successor(self, first_parent, second_parent):
        """
        Produces a "child", which increases a population as a whole by 1.

        :param first_parent: first bird, either hawk or dove
        :param second_parent: second bird, either hawk or dove
        :return: successor bird object
        """
        r = random()

        if first_parent.name == second_parent.name:
            if r >= self.mutation_rate:
                """
                two parents that belong to the same species produce the same "child" when we get a random number
                bigger than self.mutation_rate.
                """
                successor = Bird(name=first_parent.name, fitness_function=self.fitness_0)
                return successor
            elif first_parent.name == "H":  # if we got number that is less than self.mutation_rate, mutation occurs
                successor = Bird(name="D", fitness_function=self.fitness_0)
                return successor
            elif first_parent.name == "D":  # if we got number that is less than self.mutation_rate, mutation occurs
                successor = Bird(name="H", fitness_function=self.fitness_0)
                return successor

        elif first_parent.name != second_parent.name:
            fit_first_parent = (first_parent.fitness_function + 0.0001) / (first_parent.fitness_function +
                                                                           second_parent.fitness_function + 0.0001)

            if r < fit_first_parent:
                successor = Bird(name=first_parent.name, fitness_function=self.fitness_0)
                return successor
            elif r >= fit_first_parent:
                successor = Bird(name=second_parent.name, fitness_function=self.fitness_0)
                return successor

    def sort_bird_population(self, population):
        """
        Sort bird population by value of fitness functions, uses Merge Sort.

        :param population: given population for sorting
        :return: sorted population by value of fitness functions
        """
        middle = floor(len(population) / 2)

        if middle == 0:
            return population

        left_part = self.sort_bird_population(population[0:middle])
        right_part = self.sort_bird_population(population[middle:len(population)])

        for i in range(len(population) - 1, -1, -1):
            if len(left_part) > 0 and len(right_part) > 0 and \
                    left_part[len(left_part) - 1].fitness_function > right_part[len(right_part) - 1].fitness_function:
                population[i] = left_part.pop()
            elif len(right_part) > 0:
                population[i] = right_part.pop()
            else:
                population[i] = left_part.pop()

        return population

    def breed_new_individuals(self):
        """
        Produce next generation of birds by reproducing the birds that have best fitness in the current generation.

        :return: None, refills population collection
        """
        self.population = self.sort_bird_population(population=self.population)
        self.parents_list, self.cumulative_fitness_function_for_parents = [], []

        number_of_parents = floor(self.fraction_of_the_most_fit_birds * self.population_length)
        cumulative_parent_fitness = 0
        parents_fitness_functions = []

        for i in range(0, number_of_parents):
            parent = self.population[self.population_length - 1 - i]

            if parent.fitness_function < 0:
                break

            self.parents_list.append(parent)
            parents_fitness_functions.append(parent.fitness_function)
            cumulative_parent_fitness += parent.fitness_function

        for i in range(0, len(self.parents_list)):
            # normalize
            parents_fitness_functions[i] /= cumulative_parent_fitness

        self.cumulative_fitness_function_for_parents.append(0)
        for i in range(0, len(self.parents_list)):
            self.cumulative_fitness_function_for_parents.append(self.cumulative_fitness_function_for_parents[i] +
                                                                parents_fitness_functions[i])

        if len(self.parents_list) < 2:
            print("We don't have enough fit parents.")

        number_of_successors = self.population_length - len(self.parents_list)
        successors = []

        for i in range(0, number_of_successors):
            first_parent_idx, second_parent_idx = self.select_two_distinct_parent_indexes()
            first_parent, second_parent = self.parents_list[first_parent_idx], self.parents_list[second_parent_idx]

            successor = self.produce_successor(first_parent, second_parent)
            successors.append(successor)

        self.population = []

        for i in range(0, len(self.parents_list)):
            self.parents_list[i].fitness_function = self.fitness_0
            self.population.append((self.parents_list[i]))

        for i in range(0, len(successors)):
            self.population.append(successors[i])

    def run(self, number_of_rounds):
        """
        Runs the evolutionary game simulation.

        :return: None, prints results
        """
        self.initialize_population()

        for i in range(0, number_of_rounds):
            hawk_ratio = self.hawk_current_ratio

            self.hawk_ratio_list.append(hawk_ratio)
            print("iteration = {}, hawk / population = {}".format(i + 1, hawk_ratio))

            self.game()
            self.breed_new_individuals()

        print("Mean hawk ratio is: {}".format(np.mean(self.hawk_ratio_list)))

# Glavni program (pokretanje simulacije)

In [ ]:
if __name__ == "__main__":
    simulation = Simulation(population_length=1000, fitness_0=10000, mutation_rate=0.0001, hawk_probability=0.5,
                            number_of_two_bird_encounters_per_generation=10, fraction_of_the_most_fit_birds=0.5,
                            V=1, C=5)

    simulation.run(number_of_rounds=100)

iteration = 1, hawk / population = 0.515
iteration = 2, hawk / population = 0.081
iteration = 3, hawk / population = 0.125
iteration = 4, hawk / population = 0.168
iteration = 5, hawk / population = 0.171
iteration = 6, hawk / population = 0.175
iteration = 7, hawk / population = 0.188
iteration = 8, hawk / population = 0.2
iteration = 9, hawk / population = 0.213
iteration = 10, hawk / population = 0.176
iteration = 11, hawk / population = 0.187
iteration = 12, hawk / population = 0.19
iteration = 13, hawk / population = 0.191
iteration = 14, hawk / population = 0.203
iteration = 15, hawk / population = 0.224
iteration = 16, hawk / population = 0.263
iteration = 17, hawk / population = 0.265
iteration = 18, hawk / population = 0.238
iteration = 19, hawk / population = 0.206
iteration = 20, hawk / population = 0.233
iteration = 21, hawk / population = 0.195
iteration = 22, hawk / population = 0.192
iteration = 23, hawk / population = 0.217
iteration = 24, hawk / population = 0.253
iter